##### improve domain concepts 

In [1]:
import pickle
import os
import time
from datetime import datetime, date
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from rake_nltk import Metric, Rake
from collections import Counter
import re
import spacy
from nltk.corpus import wordnet
import random


##### store

In [ ]:
concept_folder="concept_seperate"
## finish all 
all_concepts_file = os.path.join(concept_folder,'all_concepts.pkl')  # edges
with open(all_concepts_file, "rb") as output_file:
    all_concepts=pickle.load(output_file)
    
## remove repeated concepts
unique_concepts = list(set(all_concepts))
concepts_path='full_concepts_form_openalex.txt'
f = open(concepts_path, "a")
for ii in range(len(unique_concepts)):
    f.write(unique_concepts[ii]+'\n')
f.close()

##### read the concepts file

In [2]:
 
if os.path.exists(concepts_path):
    # open the existing file for reading   
    with open(concepts_path, "r") as f:
        modify_full_concept_list = [line.rstrip() for line in f.readlines()]
    
    now_time = datetime.now()
    formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
    print("{}; Concepts: {:d} ".format(formatted_time,len(modify_full_concept_list)))

17-04-2023 12:06:38; Concepts: 80675 


##### filter concepts

In [3]:

starting_time = time.time()

filter_concept_any=['held','equal','dramatic','slowing','excited','occupied','charged','moving','layer','bi','argument','intuition','experiment','entirely','essentially','built','necessary','take','applicable','employ','visit','visited','herein','facilitates','varying','overlapping','addressed','issues','related','add','adds','dominant','preserve','preserves','preserved','stabilizing','match','manipulating','emerging','processed','data','continuously','analytically','argue','smoothly','connect','connects','connecting','software','matlab','toolbox','standard','industrial','technology','success','equipment','call','analogous','sense','persist','persists','throughout','calculated','useful','difficult','proved']

filter_concept_start=['sophisticated','precise','remarkably','consists','gradually','simplified','complete','techniques','partially','presented','iterative','simple','preparation','clear','priori','ae','substantial','sending','protecting','optimized','optimize','optimizing','transmits','transmit','transmitting','transmitted','processing','pre','collect','collected','measured','varied','operating','algorithms','algorithm','robustly','shall','concept','packing','successful','apparent','apparently','readily','adapted','todays','imperfect','seemingly','seeming','shelf','properties','mechanism','phenomenon','behavior','theorem','procedure','usual','form','later','calculating','fundamentally']

filter_concept_end=['illustrates','setup','consisting','set','capable','configuration','complete','borrowed','permit','utilizes','referred','refer','capable','pave','stem','preparation','scheme','optimizes','transmitted','transmit','operating','relate','packed','packing','platform','industry','adapt','adapts','adapted','arrangement','era','device','arrange','arranged','content','procedure','outlined','form','formed','followed','following','calculation']


concept_to_remove_pair=['self']
concept_to_keep_pair=['stabilization']

conditioned_filter_concept_any5=['open']
conditioned_filter_concept_any3=['driven','component']
conditioned_filter_concept_any2=['probe','inspired','technique','open','added','transfer','connected','element','exchange']

conditioned_filter_concept_start2=['doubly','probe']
conditioned_filter_concept_end2=[]

forbidden_continued_strings=['complete measurement','exact numerical','numerical technique','numerical method','complete set','pure entangled','quantum entangled','high fidelity']

improve_full_concept_list=[]

for one_concept in modify_full_concept_list:
    
    separated_words=one_concept.split()
    do_remove=0
    for word in separated_words:
        if word in filter_concept_any:
            do_remove=1
            break
        
        if len(separated_words)<5: ## only for 5 words
            if word in conditioned_filter_concept_any5:
                do_remove=1
                break

            if len(separated_words)<=3:
                if word in conditioned_filter_concept_any3:
                    do_remove=1
                    break
                
                if len(separated_words)==2: ## only for 2 words
                    if word in conditioned_filter_concept_any2:
                        do_remove=1
                        break

             
    
    if separated_words[0] in filter_concept_start:
            do_remove=1
    if separated_words[-1] in filter_concept_end:
            do_remove=1
                
    if len(separated_words)==2:
        if separated_words[0] in conditioned_filter_concept_start2: #check the start word 
            do_remove=1
        if separated_words[-1] in conditioned_filter_concept_end2: #check the last word 
            do_remove=1

    if do_remove==0:
        for word in forbidden_continued_strings:
            if word in one_concept:
                do_remove=1
                break

    if do_remove==0:
        improve_full_concept_list.append(one_concept)
        
print("Concepts: {:d} ; Store: {:d}; Remove: {:d} ".format(len(modify_full_concept_list), len(improve_full_concept_list),len(modify_full_concept_list)-len(improve_full_concept_list)))
elapsed_time = time.time() - starting_time
print("Elapsed time: {:.2f} seconds".format(elapsed_time))

now_time =  datetime.now()
formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
print("{}; Concepts: {:d} ".format(formatted_time,len(improve_full_concept_list)))

Concepts: 80675 ; Store: 80642; Remove: 33 
Elapsed time: 0.34 seconds
17-04-2023 12:06:46; Concepts: 80642 


##### restore the file

In [4]:

# Delete the orginal txt and re-create a new one with the improved concepts 
if os.path.exists(concepts_path):
    os.remove(concepts_path)
    print("txt has been deleted.")

    # re-Create the text file  
    f = open(concepts_path, "a")
    for ii in range(len(improve_full_concept_list)):
        f.write(improve_full_concept_list[ii]+'\n')
    f.close()
    print("re-create text and store information.")  
else:
    f = open(concepts_path, "a")
    for ii in range(len(improve_full_concept_list)):
        f.write(improve_full_concept_list[ii]+'\n')
    f.close()
    print("create text and store information.")
    
now_time = datetime.now()
formatted_time = now_time.strftime("%d-%m-%Y %H:%M:%S")
print("{}; Concepts: {:d} ".format(formatted_time,len(improve_full_concept_list)))


txt has been deleted.
re-create text and store information.
17-04-2023 12:06:52; Concepts: 80642 


##### additionally, store a pkl file (as a backup)

In [ ]:
concepts_path_pkl=os.path.join(concepts_folder_to_use,'improved_concepts_form_openalex.pkl')
with open(concepts_path_pkl, "wb") as output_file:
    pickle.dump(improve_full_concept_list, output_file)